In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

In [2]:
def features(df):
    words_ru = ['порн', 'очко', 'ебет', 'раком', 'секс', 'лесби',
             'куни', 'голы', 'эроти', 'раздетый', 'соски',
             'обнаженн', 'интим', 'видео', 'гей', 'орал', 'анал', 'еб', 'трах', 'жар', 'стринг','жоп','конч',
                'вагин', 'попа', 'попу', 'попк', 'пизд', 'орги', 'фистинг', 'сперм']
    words_en = ["porn", "sex", "x", "xxx", "lesbi", "fuck",  "horny", "naked", "dirty"
               "gay", "vagin", "oral"
               "nude", "tit", "dick", "girl", "hot", "undress", "anal", "slut", "topless", "pussy",
                "video", "ero", "ass", "cum", "sperm"]

    title = df["title"].values
    url = df["url"].values

    result = []
    for i, val in enumerate(title):
        result.append([])
        for j in words_ru:
            result[-1].append(int(j in val.lower()))
        for j in words_en:
            result[-1].append(int(j in val.lower()))
        for j in words_en:
            result[-1].append(int(j in url[i].lower()))
    return result

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
X_train = train_df["title"].values + ' ' + train_df["url"].values
X_test = test_df["title"].values+ ' '+test_df["url"].values
y_train = train_df["target"].astype(int).values

In [5]:
import nltk

stop_words = nltk.corpus.stopwords.words('russian')

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
ans=[]
i=0.1
while i<0.5:
    vectorizer = CountVectorizer(stop_words=stop_words,ngram_range=(2,4),max_df=i)
    model = SGDClassifier(class_weight='balanced')

    X_train_vectorized = sparse.hstack((vectorizer.fit_transform(X_train), sparse.csr_matrix(np.asarray(features(train_df)))))
    model.fit(
        X_train_vectorized,
        y_train
    )
    y_pred = model.predict(
        X_train_vectorized
    )
    ans.append(f1_score(y_train, y_pred))
    print(i)
    i+=0.1
print(ans)

0.1
0.2
0.30000000000000004
0.4
[0.9976705292079798, 0.9977003255383329, 0.997789593165661, 0.9976404527941221]


In [7]:
import re


def preprocessor(text):
    whitespaced_text = re.sub("[а-яa-z]", ' ', text.lower())
    return re.sub(' +', ' ',  whitespaced_text)

In [8]:
# на самом деле на тестовых данных в kagle модель работает лучше с параметрами по умолчанию. Но наибольший скор на трейне выдаётся именно при таких параметрах. С препроцессором тоже игрался, но с ним прям всё супер плохо.

vectorizer = CountVectorizer(ngram_range=(2,4),max_df=0.4)
model = SGDClassifier(class_weight='balanced')

X_train_vectorized = sparse.hstack((vectorizer.fit_transform(X_train), sparse.csr_matrix(np.asarray(features(train_df)))))
model.fit(
    X_train_vectorized,
    y_train
)
y_pred = model.predict(
    X_train_vectorized
)
print(f1_score(y_train, y_pred))

0.9978791409027092


In [9]:
X_test_vectorized = sparse.hstack((vectorizer.transform(X_test), sparse.csr_matrix(np.asarray(features(test_df)))))

test_df["target"] = model.predict(X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)